# ANALISE - Bike Sharing

### Informações dos atributos do dataset Bike Sharing
[Fonte](http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset)

Tanto hour.csv quanto day.csv têm os seguintes campos, exceto hr, que não está disponível em day.csv

- **instant**: índice de registro
- **dteday** : data
- **season** : estação (1: inverno, 2: primavera, 3: verão, 4: outono)
- **yr**: ano (0: 2011, 1: 2012)
- **mnth**: mês (1 a 12)
- **hr** : hora (0 a 23)
- **holiday**  : se o dia é feriado ou não (extraído de [Web Link])
- **weekday** : dia da semana
- **workingday** : se o dia não for nem fim de semana nem feriado é 1, caso contrário é 0.
- **weathersit**: condições climáticas
    - 1: Claro, Poucas nuvens, Parcialmente nublado, Parcialmente nublado
    - 2: Névoa + Nublado, Névoa + Nuvens quebradas, Névoa + Poucas nuvens, Névoa
    - 3: Neve fraca, Chuva fraca + Trovoada + Nuvens dispersas, Chuva fraca + Nuvens dispersas
    - 4: Chuva Forte + Paletes de Gelo + Tempestade + Névoa, Neve + Neblina
- **temp** : Temperatura normalizada em Celsius. Os valores são derivados via (t-t_min)/(t_max-t_min), t_min=-8, t_max=+39 (somente na escala horária)
- **atemp**: Sensação térmica normalizada em Celsius. Os valores são derivados via (t-t_min)/(t_max-t_min), t_min=-16, t_max=+50 (somente na escala horária)
- **hum**: Umidade normalizada. Os valores são divididos por 100 (máx)
- **windspeed**: velocidade do vento normalizada. Os valores são divididos por 67 (máx)
- **casual**: contagem de usuários casuais
- **registered**: contagem de usuários registrados
- **cnt**: contagem do total de bicicletas alugadas, incluindo bicicletas casuais e registradas

## dbutils

In [ ]:
#CRIANDO PASTA DE DESTINO
dbutils.fs.mkdirs('/FileStore/tables/aula-databricks/projeto_final/bikeSharing/')

In [ ]:
#COPIANDO ARQUIVOS DA ORIGEM PARA O DESTINO 
dbutils.fs.cp(
    '/databricks-datasets/bikeSharing/', 
    '/FileStore/tables/aula-databricks/projeto_final/bikeSharing/', 
    recurse=True
)

In [ ]:
#VERIFICANDO INFORMAÇÕES DA PASTA
display(dbutils.fs.ls('/FileStore/tables/aula-databricks/projeto_final/bikeSharing/'))

## SQL

In [ ]:
#CRIANDO UM DATABASE PARA ARMAZENAR O DADOS
%sql
CREATE DATABASE IF NOT EXISTS projeto_final

In [ ]:
#VISUALIZANDO BANCO DE DADOS EXISTENTES
%sql
SHOW DATABASES

In [ ]:
#USANDO O BANCO DE DADOS QUE CRIAMOS
%sql
USE projeto_final

- O dataset day.csv possui o seguinte conjunto de atributos:

| nome | tipo |
|------|------|
| instant | int |
| dteday | timestamp |
| season | int |
| yr | int |
| mnth | int |
| holiday | int |
| weekday | int |
| workingday | int |
| weathersit | int |
| temp | float |
| atemp | float |
| hum | float |
| windspeed | float |
| casual | int |
| registered | int |
| cnt | int |

In [ ]:
#CRIANDO E CARREGANDO UMA TABELA CONFORME O ARQUIVO "DAY" DO DBFS

%sql
CREATE TABLE dados_dia(
    instant int,
    dteday timestamp,
    season int,
    yr int,
    mnth int,
    holiday int,
    weekday int,
    workingday int,
    weathersit int,
    temp float,
    atemp float,
    hum float,
    windspeed float,
    casual int,
    registered int,
    cnt int
  )
  USING CSV
  OPTIONS (
    path '/FileStore/tables/aula-databricks/projeto_final/bikeSharing/data-001/day.csv',
    header 'true',
    delimiter ','
  )

In [ ]:
#VERIFICANDO TABELA CRIADA
%sql
SHOW TABLES

In [ ]:
#VISUALIZANDO DADOS CARREGADOS
%sql
SELECT *
  FROM dados_dia
    LIMIT 10

- O dataset hour.csv possui o seguinte conjunto de atributos:

| nome | tipo |
|------|------|
| instant | int |
| dteday | timestamp |
| season | int |
| yr | int |
| mnth | int |
| hr | int |
| holiday | int |
| weekday | int |
| workingday | int |
| weathersit | int |
| temp | float |
| atemp | float |
| hum | float |
| windspeed | float |
| casual | int |
| registered | int |
| cnt | int |

In [ ]:
#CRIANDO E CARREGANDO UMA TABELA CONFORME O ARQUIVO "HOUR" DO DBFS
%sql
CREATE TABLE dados_hora(
    instant int,
    dteday timestamp,
    season int,
    yr int,
    mnth int,
    hr int,
    holiday int,
    weekday int,
    workingday int,
    weathersit int,
    temp float,
    atemp float,
    hum float,
    windspeed float,
    casual int,
    registered int,
    cnt int
  )
  USING CSV
  OPTIONS (
    path '/FileStore/tables/aula-databricks/projeto_final/bikeSharing/data-001/hour.csv',
    header 'true',
    delimiter ','
  )

In [ ]:
#VERIFICANDO TABELA CRIADA

%sql
SHOW TABLES

In [ ]:
#VALIDANDO DADOS

%sql
SELECT *
  FROM dados_hora
    LIMIT 10

In [ ]:
#TOTAL DE USUARIO CASUAL E REGISTRADO POR ESTAÇÃO DO ANO
%sql
SELECT season, SUM (casual) AS casual_total, SUM (registered) AS registered_total
  FROM dados_dia
    GROUP BY season
      ORDER BY season 

In [ ]:
#TOTAL DE USUARIOS POR HORA
%sql
SELECT hr, SUM (cnt) AS alugueis_total
  FROM dados_hora
    GROUP BY hr
      ORDER BY hr

## Spark SQL

In [ ]:
#CRIANDO NOVA TABELA FILTRANDO APENAS NO ANOR DE 2011
temp_view = spark.sql(''' 
    SELECT *
        FROM dados_dia
            WHERE yr == 2011
''')


temp_view.createOrReplaceTempView('DADOS_DIA_2011')

In [ ]:
#SELECIONANDO ALUGUEIS POR CLIMA E DIA DA SEMANA
spark.sql('''
    SELECT weathersit, workingday, SUM (cnt) AS alugueis_total 
        FROM DADOS_DIA_2011
            GROUP BY weathersit, workingday
                ORDER BY weathersit, workingday
''').show()